In [1]:
import numpy as np
import pandas as pd
from dataset import MovieLensDataset
from sklearn.decomposition import PCA
from preprocessing import Preprocessing
from clustering import Clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from typing import List
from predict import predict
from sklearn.metrics import mean_squared_error

In [2]:
dataset_name = 'ml-latest-small'
dataset = MovieLensDataset(dataset_name)
preprocessing = Preprocessing(dataset)
clustering = Clustering(preprocessing)

In [6]:
ratings = dataset.get_ratings().drop('timestamp', axis=1)
ratings.head(50)

rating
userId movieId        
1      1           4.0
       3           4.0
       6           4.0
       47          5.0
       50          5.0
       70          3.0
       101         5.0
       110         4.0
       151         5.0
       157         5.0
       163         5.0
       216         5.0
       223         3.0
       231         5.0
       235         4.0
       260         5.0
       296         3.0
       316         3.0
       333         5.0
       349         4.0
       356         4.0
       362         5.0
       367         4.0
       423         3.0
       441         4.0
       457         5.0
       480         4.0
       500         3.0
       527         5.0
       543         4.0
       552         4.0
       553         5.0
       590         4.0
       592         4.0
       593         4.0
       596         5.0
       608         5.0
       648         3.0
       661         5.0
       673         3.0
       733         4.0
       736         3.0
       780         3.0
       804         4.0
       919         5.0
       923         5.0
       940         5.0
       943         4.0
       954         5.0
       1009        3.0

: 

In [4]:
movies = dataset.get_movies()
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.index.get_level_values('userId').unique()

Index([     1,      2,      3,      4,      5,      6,      7,      8,      9,
           10,
       ...
       330966, 330967, 330968, 330969, 330970, 330971, 330972, 330973, 330974,
       330975],
      dtype='int64', name='userId', length=330975)

In [6]:
movie_cluster = clustering.get_movie_cluster(1, 30)

In [7]:
users = preprocessing.preprocess_users(movie_cluster)

,0,1,2,3,4
userId,,,,,
1,1.304019,-0.252111,-0.044110,0.147171,-0.044369
2,2.223970,-1.467558,0.892294,-0.754733,0.117547
7,-0.781634,-0.448102,-0.887338,-0.232842,0.318219
9,-0.824079,-0.614435,0.586111,-0.316880,-0.012935
10,-1.428881,0.015411,0.056082,0.159266,-0.005049


In [3]:
user_cluster = clustering.get_user_cluster(1, 1, user_n_clusters=8, movie_n_clusters=30)

In [8]:
ratings = dataset.get_ratings()
ratings[ratings.index.get_level_values('movieId') == 1 & ratings.index.get_level_values('userId').isin(user_cluster)]['rating'].mean()

rating   timestamp
userId movieId                    
1      1           4.0  1225734739
       110         4.0  1225865086
       158         4.0  1225733503
       260         4.5  1225735204
       356         5.0  1225735119

In [2]:
predict('ml-latest-small', 1, 1)

KeyError: 1

In [3]:
y_true = []
y_pred_model = []
y_pred_base = []

for (user_id, movie_id), (rating, _) in dataset.get_ratings().sample(100).iterrows():
    prediction = predict('ml-latest-small', user_id, movie_id)
    y_true.append(rating)
    y_pred_model.append(prediction)
    y_pred_base.append(3.5)

print('Model:', mean_squared_error(y_true, y_pred_model))
print('Baseline:', mean_squared_error(y_true, y_pred_base))

TypeError: cannot unpack non-iterable numpy.float64 object

In [20]:
def mypredictor(user_id, movie_id):

    ratings = dataset.get_ratings().drop('timestamp', axis=1)
    mean_rating = ratings['rating'].mean()
    users = dataset.get_ratings().drop('timestamp', axis=1)
    users = ratings.join(preprocessing.movies_ohe()).apply(
        lambda row: (row * row['rating']).replace(0, pd.NA), axis=1
    ).drop('rating', axis=1)

    for user_id in ratings.index.get_level_values('userId').unique():
        users.loc[(user_id, -1), :] = np.repeat(mean_rating, ratings.shape[1])

    users = Preprocessing.standarize(ratings.groupby(level=0).mean())

    user_cluster = Clustering.get_cluster(ratings, user_id)

    return ratings[ratings.index.get_level_values('movieId') == 1 & ratings.index.get_level_values('userId').isin(user_cluster)]['rating'].mean()

In [21]:
user_id = 1


[1,
 17,
 25,
 30,
 43,
 46,
 52,
 53,
 69,
 80,
 93,
 106,
 119,
 122,
 128,
 154,
 169,
 171,
 186,
 227,
 246,
 251,
 276,
 282,
 319,
 336,
 337,
 348,
 362,
 371,
 376,
 400,
 413,
 417,
 441,
 452,
 459,
 460,
 475,
 491,
 515,
 523,
 533,
 538,
 543,
 573,
 581,
 584,
 586,
 601]